このファイル上で、訓練済みモデルを用いてMIDI生成を行い、モデルの性能を評価する

inputを何にするかは未定（txt, seed, midi...）

input_midiファイルのトークナイズ
とりま　pitch と　duration だけにしてモデルを完成させてからいじる

midi解析にはmusic21ではなく、midoを使う

In [1]:
import torch


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

2.5.1+cu118
True
1


In [4]:
#モデルを訓練
%run model_transformer.py

is_cuda: True
MIDIgen(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (LSTM): LSTM(4, 256, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (ReLU): ReLU()
)


In [5]:
#以下のセルはモデルに関係ない、確認用などのコード
#gpuがつかえるか確認
import time
# timeit: テンソルを二乗し、計算にかかった時間を表示する。
def timeit(x):
    t0 = time.time()
    x = torch.mm(x, x)
    dt = time.time() - t0
    return dt

#GPUで計算
# 100×100のランダムな行列を二乗する。
time_cuda_10k = 0.000
for i in range(10):
    x = torch.rand(10000,10000).to('cuda')
    time_cuda_10k += timeit(x)

#CPUで計算
time_cpu_10k = 0.000
for i in range(10):
    x = torch.rand(10000,10000).to('cpu')
    time_cpu_10k += timeit(x)

print(f"GPU:\n10000x10000: {time_cuda_10k:.3f}")
print(f"CPU:\n10000x10000: {time_cpu_10k:.3f}")

GPU:
10000x10000: 0.243
CPU:
10000x10000: 67.820


In [6]:
from music21 import converter

# MIDIファイルを読み込む
midi_file = converter.parse('./data/output/generated_output3.mid')

# 詳細な情報を出力
print(midi_file)

# 各音符の情報を表示
for element in midi_file.recurse().notesAndRests:
    print(element)

<music21.stream.Score 0x220b5e060f0>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note C#>
<music21.note.Note F#>
<music21.note.Note C#>
<music21.note.Note G>
<music21.note.Note G#>
<music21.note.Note A>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note E>
<music21.note.Note G>
<music21.note.Note B>
<music21.note.Note B>
<music21.note.Note A>
<music21.note.Note E->
<music21.note.Note D>
<music21.note.Note E->
<music21.note.Note E>
<music21.note.Note B>
<music21.note.Note F#>
<music21.note.Note F#>
<music21.note.Note D>
<music21.note.Note B>
<music21.note.Note A>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C#>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note F#>
<music21.note.Note G>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Note C#>
<music21.note.Note D>
<mus

In [21]:
import mido

# MIDIファイルを開く
mid = mido.MidiFile('./data/midi_data/mz_331_1.mid', clip=True)

for track in mid.tracks:
    print(len(track))

# MIDIトラックの情報を表示
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: ')
    for msg in track:
        print(msg.bytes())


#最初の６行はメタデータ？消してもモデルには影響ないはず
#各行先頭の255はメタイベントを示すので、これも消してもいい

2183
7403
5053
1325
2
2
2
2
2
2
Track 0: 
[255, 3, 27, 75, 108, 97, 118, 105, 101, 114, 115, 111, 110, 97, 116, 101, 32, 78, 114, 46, 32, 49, 49, 32, 75, 86, 32, 51, 51, 49]
[255, 2, 35, 67, 111, 112, 121, 114, 105, 103, 104, 116, 32, 169, 32, 50, 48, 48, 53, 32, 118, 111, 110, 32, 66, 101, 114, 110, 100, 32, 75, 114, 252, 103, 101, 114, 46, 32]
[255, 1, 14, 65, 109, 97, 100, 101, 117, 115, 32, 77, 111, 122, 97, 114, 116]
[255, 1, 28, 70, 101, 114, 116, 105, 103, 103, 101, 115, 116, 101, 108, 108, 116, 32, 97, 109, 32, 32, 49, 46, 49, 46, 50, 48, 48, 54, 10]
[255, 1, 19, 85, 112, 100, 97, 116, 101, 32, 97, 109, 32, 54, 46, 55, 46, 51, 48, 49, 51, 10]
[255, 1, 21, 68, 97, 117, 101, 114, 58, 32, 49, 51, 58, 53, 50, 32, 77, 105, 110, 117, 116, 101, 110, 10]
[255, 88, 4, 6, 3, 12, 8]
[255, 89, 2, 3, 0]
[255, 81, 3, 14, 106, 93]
[255, 6, 9, 84, 104, 101, 109, 97, 32, 49, 58, 49]
[255, 81, 3, 13, 1, 170]
[255, 81, 3, 15, 138, 135]
[255, 81, 3, 14, 106, 93]
[255, 81, 3, 16, 100, 130]
[255, 81